In [3]:
"""
This script is intended to help with debugging a model.
It imports a model, initializes a model object with given factors,
sets up pseudorandom number generators, and runs one or more replications.
"""

import sys
import os.path as o
import simopt
import mrg32k3a
#sys.path.append(o.abspath(o.join(o.dirname(sys.modules[__name__].__file__), "..")))
import numpy as np
import matplotlib.pyplot as plt

# Import random number generator.
from mrg32k3a.mrg32k3a import MRG32k3a

# Import model.

# from models.<filename> import <model_class_name>
# Replace <filename> with name of .py file containing model class.
# Replace <model_class_name> with name of model class.

# Fix factors of model. Specify a dictionary of factors.

# fixed_factors = {}  # Resort to all default values.
# Look at Model class definition to get names of factors.

# Initialize an instance of the specified model class.

# mymodel = <model_class_name>(fixed_factors)
# Replace <model_class_name> with name of model class.

# Working example for Dynamic Newsvendor model.
# -----------------------------------------------
from simopt.models.dynamnews import DynamNews

n = 2 # sample_size_per_system
num_products = 3
max_inventory = 10
# k = max_inventory**num_products (k = 10**3 by default)
k = 1000

profit_matrix = np.zeros((n, k))
n_prod_stockout_matrix = np.zeros((n, k))

# Other factors
num_customer = 30
c_utility = [6, 7, 8]
price = [9, 9, 9]
cost = [5, 5, 5]

ind_sampling_counter = 0
for prod1init in range(max_inventory):
    for prod2init in range(max_inventory):
        for prod3init in range(max_inventory):
            fixed_factors = {"num_prod": num_products,
                             "num_customer": num_customer,
                             "c_utility": c_utility,
                             "price": price,
                             "cost": cost,
                             "init_level": (prod1init, prod2init, prod3init)}  # This one is the decision variable. 
            # Initialize a model.
            mymodel = DynamNews(fixed_factors)
            # Initialize random number generators.
            # If doing independent sampling across systems.
            rng_list = [MRG32k3a(s_ss_sss_index=[ind_sampling_counter, ss, 0]) for ss in range(mymodel.n_rngs)]
            # If doing CRN across systems:
            # rng_list = [MRG32k3a(s_ss_sss_index=[0, ss, 0]) for ss in range(mymodel.n_rngs)]
            for rep_idx in range(n):
                # Can manually increment subsubstreams too,
                # but not necessary for this experiment.
                responses, _ = mymodel.replicate(rng_list)
                profit_matrix[rep_idx, ind_sampling_counter] = responses["profit"]
                n_prod_stockout_matrix[rep_idx, ind_sampling_counter] = responses["n_prod_stockout"]
            # Increment counter.
            ind_sampling_counter += 1
# -----------------------------------------------


print(len(profit_matrix[0]))
print(profit_matrix)

plt.plot(profit_matrix[0])

# The rest of this script requires no changes.

# Check that all factors describe a simulatable model.
# Check fixed factors individually.
# for key, value in mymodel.factors.items():
    # print(f"The factor {key} is set as {value}. Is this simulatable? {bool(mymodel.check_simulatable_factor(key))}.")
# Check all factors collectively.
# print(f"Is the specified model simulatable? {bool(mymodel.check_simulatable_factors())}.")

# Create a list of RNG objects for the simulation model to use when
# running replications.
# rng_list = [MRG32k3a(s_ss_sss_index=[0, ss, 0]) for ss in range(mymodel.n_rngs)]

# Run a single replication of the model.
# responses, gradients = mymodel.replicate(rng_list)
# print("\nFor a single replication:")
# print("\nResponses:")
# for key, value in responses.items():
#     print(f"\t {key} is {value}.")
# print("\n Gradients:")
# for outerkey in gradients:
#     print(f"\tFor the response {outerkey}:")
#     for innerkey, value in gradients[outerkey].items():
#         print(f"\t\tThe gradient w.r.t. {innerkey} is {value}.")


ModuleNotFoundError: No module named 'simopt'